In [17]:
from core.ConfigConnection import ConfigConnection
from core.Backup import Backup
from core.Tools import Tools

import mysql.connector

if __name__ == "__main__":
    config = ConfigConnection(path="./config.ini")
    connection = mysql.connector.connect(**config.getConfig())
    cursor = connection.cursor()
    t = Tools()
    
    sql_file = t.readFile('SQL', 'dml-web-scraping.sql')
    sql_script = ''.join(sql_file)
    
    try:
        # Ejecuta el script SQL
        cursor.execute(sql_script)
    
        # Confirma los cambios en la base de datos
        connection.commit()
    
        print("Script SQL ejecutado exitosamente.")
    except Exception as e:
        print("Error al ejecutar el script SQL:", e)
        connection.rollback()
        
    # Cierra el cursor y la conexión
    cursor.close()
    connection.close()

Error al ejecutar el script SQL: Commands out of sync; you can't run this command now


MySQLInterfaceError: Commands out of sync; you can't run this command now

In [13]:
cursor